In [5]:
import os
import csv

# Directory containing experiment folders
experiments_dir = "experiments"

# Collect rows for each table
parameters_rows = []  # For the parameters table
summary_rows = []     # For the summary table

# Keep track of all parameter keys seen across experiments
all_param_keys = set()

# Original summary fields in summary.csv
summary_keys = [
    "Best Iteration",
    "Best Loss",
    "Avg Gradient Time",
    "Std Gradient Time",
    "Avg Sampling Time",
    "Std Sampling Time",
    "Avg PGD Time",
    "Std PGD Time",
    "Avg Loss Time",
    "Std Loss Time",
]

# We map them to shorter column names and combine avg±std.
summary_column_map = {
    "Best Iteration": "Iter",
    "Best Loss": "Loss",
    "Avg Gradient Time": "Grad (s)",
    "Std Gradient Time": "Grad (s)",
    "Avg Sampling Time": "Sampling (s)",
    "Std Sampling Time": "Sampling (s)",
    "Avg PGD Time": "PGD (s)",
    "Std PGD Time": "PGD (s)",
    "Avg Loss Time": "LossTime (s)",
    "Std Loss Time": "LossTime (s)",
}

# Final column order for the results table
final_result_columns = [
    "Experiment",
    "Iter",
    "Loss",
    "Grad (s)",
    "Sampling (s)",
    "PGD (s)",
    "LossTime (s)",
]

def latex_escape(text: str) -> str:
    """
    Escape underscores so they don't become subscripts in LaTeX.
    Extend as needed for other special characters.
    """
    return text.replace("_", "\\_")

def format_numeric(value: str, param_name: str = "") -> str:
    """
    Attempt to parse and format:
      - If param_name == "seed", interpret as integer (no decimals).
      - Else parse as float and format with 4 decimals.
      - If parsing fails, treat as text and underscore-escape.
    """
    # Special case for booleans
    if value in ("True", "False"):
        return value

    # Special case for integer parameters
    if param_name.lower() in ("seed", "iter", "search_width", "min_search_width", "num_steps"):
        try:
            # If it parses as an integer, return it without decimals
            return str(int(float(value)))
        except ValueError:
            # If it can't parse, treat as text
            return latex_escape(value)

    # Try parsing as float with 4 decimals
    try:
        float_val = float(value)
        return f"{float_val:.4f}"
    except ValueError:
        # Not a float, so treat as text
        return latex_escape(value)

def combine_avg_std(avg_val: str, std_val: str) -> str:
    """
    Combine avg±std, each with 4-decimal formatting.
    If either avg or std is missing/'nan', show 0.00±0.00.
    """
    # If either is missing or 'nan', return "0.00±0.00"
    if (not avg_val) or (avg_val.lower() == "nan") or \
       (not std_val) or (std_val.lower() == "nan"):
        return "0.00±0.00"
    
    # Format each with 4 decimals
    try:
        avg_float = float(avg_val)
        std_float = float(std_val)
        return f"{avg_float:.4f}±{std_float:.4f}"
    except ValueError:
        # If parsing fails, fallback to "0.00±0.00"
        return "0.00±0.00"

# Gather the experiment folders in a sorted order
experiment_folders = sorted(
    folder for folder in os.listdir(experiments_dir)
    if os.path.isdir(os.path.join(experiments_dir, folder))
)

# Process each experiment folder and assign IDs (exp1, exp2, ...)
for i, folder in enumerate(experiment_folders, start=1):
    exp_id = f"exp{i}"
    folder_path = os.path.join(experiments_dir, folder)

    # ------------------ PARAMETERS TABLE ------------------
    parameters = {}
    params_file = os.path.join(folder_path, "parameters.csv")
    if os.path.exists(params_file):
        with open(params_file, newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                param = row["Parameter"]
                value = row["Value"]
                parameters[param] = value
                all_param_keys.add(param)

    param_row = {"Experiment": exp_id}
    param_row.update(parameters)
    parameters_rows.append(param_row)

    # ------------------ RESULTS TABLE ------------------
    summary_file = os.path.join(folder_path, "summary.csv")
    summary = {}
    if os.path.exists(summary_file):
        with open(summary_file, newline='') as csvfile:
            reader = csv.reader(csvfile)
            header = next(reader, None)
            data = next(reader, None)
            if header and data:
                summary_dict = dict(zip(header, data))
                for key in summary_keys:
                    summary[key] = summary_dict.get(key, "")
    # Build a new row with combined columns
    summary_row = {"Experiment": exp_id}

    # (1) Best Iteration -> "Iter"
    iteration_val = summary.get("Best Iteration", "")
    summary_row["Iter"] = format_numeric(iteration_val)

    # (2) Best Loss -> "Loss"
    loss_val = summary.get("Best Loss", "")
    summary_row["Loss"] = format_numeric(loss_val)

    # (3) Grad (s): combine "Avg Gradient Time" + "Std Gradient Time"
    summary_row["Grad (s)"] = combine_avg_std(
        summary.get("Avg Gradient Time", ""),
        summary.get("Std Gradient Time", "")
    )

    # (4) Sampling (s): combine "Avg Sampling Time" + "Std Sampling Time"
    summary_row["Sampling (s)"] = combine_avg_std(
        summary.get("Avg Sampling Time", ""),
        summary.get("Std Sampling Time", "")
    )

    # (5) PGD (s): combine "Avg PGD Time" + "Std PGD Time"
    summary_row["PGD (s)"] = combine_avg_std(
        summary.get("Avg PGD Time", ""),
        summary.get("Std PGD Time", "")
    )

    # (6) LossTime (s): combine "Avg Loss Time" + "Std Loss Time"
    summary_row["LossTime (s)"] = combine_avg_std(
        summary.get("Avg Loss Time", ""),
        summary.get("Std Loss Time", "")
    )

    summary_rows.append(summary_row)

# ----------------------------------------------------------------------------
# 1) TABLE FOR PARAMETERS
# ----------------------------------------------------------------------------
param_columns = ["Experiment"] + sorted(all_param_keys, key=lambda x: x.lower())

latex_params = (
    "\\begin{table}[ht]\n"
    "\\centering\n"
    "\\resizebox{\\textwidth}{!}{%\n"  # <-- Wrap in \resizebox
    "  \\begin{tabular}{" + "l" * len(param_columns) + "}\n"
    "  \\hline\n"
)

# Header row (escape underscores in column names)
escaped_headers = [latex_escape(col) for col in param_columns]
latex_params += "  " + " & ".join(escaped_headers) + " \\\\\n"
latex_params += "  \\hline\n"

# Rows
for row in parameters_rows:
    row_values = []
    for col in param_columns:
        val = row.get(col, "")
        val_formatted = format_numeric(val, param_name=col)
        row_values.append(val_formatted)
    latex_params += "  " + " & ".join(row_values) + " \\\\\n"

latex_params += (
    "  \\hline\n"
    "  \\end{tabular}\n"
    "} % end of resizebox\n"
    "\\caption{Experiment parameters.}\n"
    "\\label{tab:experiment_parameters}\n"
    "\\end{table}\n"
)

# ----------------------------------------------------------------------------
# 2) TABLE FOR RESULTS
# ----------------------------------------------------------------------------
latex_summary = (
    "\\begin{table}[ht]\n"
    "\\centering\n"
    "\\resizebox{\\textwidth}{!}{%\n"  # <-- Wrap in \resizebox
    "  \\begin{tabular}{" + "l" * len(final_result_columns) + "}\n"
    "  \\hline\n"
)

# Header row
latex_summary += "  " + " & ".join(final_result_columns) + " \\\\\n"
latex_summary += "  \\hline\n"

# Rows
for row in summary_rows:
    row_values = []
    for col in final_result_columns:
        val = row.get(col, "")
        # We already combined times into "0.00±0.00" or "avg±std",
        # so just ensure final numeric formatting if it's a single float or integral
        # For times, we keep them as the combined string. No further splitting needed.
        if col in ["Grad (s)", "Sampling (s)", "PGD (s)", "LossTime (s)"]:
            # These are already combined strings, just do underscore escaping
            val_formatted = latex_escape(val)
        else:
            val_formatted = format_numeric(val, param_name=col)
        row_values.append(val_formatted)
    latex_summary += "  " + " & ".join(row_values) + " \\\\\n"

latex_summary += (
    "  \\hline\n"
    "  \\end{tabular}\n"
    "} % end of resizebox\n"
    "\\caption{Summary of experiment results.}\n"
    "\\label{tab:experiment_summary}\n"
    "\\end{table}\n"
)

# Print both tables
print("%% ========== TABLE: PARAMETERS ========== %%")
print(latex_params)
print("\n%% ========== TABLE: SUMMARY ========== %%")
print(latex_summary)


%% ========== TABLE: PARAMETERS ========== %%
\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{%
  \begin{tabular}{lllllllllll}
  \hline
  Experiment & alpha & debug\_output & dynamic\_search & eps & gcg\_attack & min\_search\_width & num\_steps & pgd\_attack & search\_width & seed \\
  \hline
  exp1 & 0.0078 & True & False & 0.2510 & False & 0 & 600 & True & 0 & 1 \\
  exp2 & 0.0078 & False & False & 0.2510 & True & 512 & 250 & False & 512 & 1 \\
  exp3 & 0.0078 & False & False & 0.2510 & True & 512 & 250 & True & 512 & 1 \\
  exp4 & 0.0078 & False & True & 0.2510 & True & 32 & 250 & True & 512 & 1 \\
  \hline
  \end{tabular}
} % end of resizebox
\caption{Experiment parameters.}
\label{tab:experiment_parameters}
\end{table}


%% ========== TABLE: SUMMARY ========== %%
\begin{table}[ht]
\centering
\resizebox{\textwidth}{!}{%
  \begin{tabular}{lllllll}
  \hline
  Experiment & Iter & Loss & Grad (s) & Sampling (s) & PGD (s) & LossTime (s) \\
  \hline
  exp1 & 589 & 0.0338 & 0.4413±